# ETA Estimator with Ban Areas

This notebook calculates estimated time of arrival (ETA) considering temporal ban areas along the route.

## Requirements:
- OpenRouteService API key
- `ban_times.csv` file in the same directory
- Required Python packages: `pandas`, `openrouteservice`, `folium`, `dateutil`

In [ ]:
# Import required libraries
import os
import pandas as pd
import openrouteservice
from datetime import datetime, timedelta, time as dt_time
from dateutil import tz
import math
import folium
import json
from IPython.display import display, HTML, IFrame

# Constants
BAN_CSV = "ban_times.csv"
BAN_RADIUS_KM = 20
SAUDI_TZ = tz.gettz('Asia/Riyadh')
DEFAULT_SPEED_KMPH = 60.0

print("✅ Libraries imported successfully!")

In [ ]:
def haversine(lat1, lon1, lat2, lon2):
    """Calculate the great circle distance between two points on Earth (in km)."""
    R = 6371
    phi1, phi2 = math.radians(lat1), math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlambda = math.radians(lon2 - lon1)
    a = math.sin(dphi / 2) ** 2 + math.cos(phi1) * math.cos(phi2) * math.sin(dlambda / 2) ** 2
    return 2 * R * math.asin(math.sqrt(a))

def parse_time(tstr):
    """Parse time string (e.g., '6:00') into datetime.time object."""
    h, m = map(int, str(tstr).split(":"))
    return dt_time(h, m)

def point_in_ban_area(lat, lon, ban_lat, ban_lon, radius_km):
    """Check if a point is within the ban area radius."""
    return haversine(lat, lon, ban_lat, ban_lon) <= radius_km

def load_ban_areas(csv_path):
    """Load ban areas from CSV into a DataFrame."""
    if not os.path.exists(csv_path):
        raise FileNotFoundError(f"Ban areas CSV file not found: {csv_path}")
    return pd.read_csv(csv_path)

def get_route_from_ors(client, start_lat, start_lon, end_lat, end_lon):
    """Get route from OpenRouteService API."""
    coords = [(start_lon, start_lat), (end_lon, end_lat)]
    try:
        route = client.directions(coords, profile='driving-car', format='geojson', instructions=False)
        return route
    except Exception as e:
        raise Exception(f"Error fetching route from OpenRouteService: {e}")

print("✅ Helper functions defined!")

In [ ]:
def calculate_eta_with_bans(start_lat, start_lon, end_lat, end_lon, start_datetime, ors_api_key, 
                           vehicle_key=None, key=None, ban_radius_km=None, vehicle_speed_kmph=None):
    """Calculate ETA considering ban areas along the route."""
    ban_df = load_ban_areas(BAN_CSV)
    
    if ban_radius_km is None:
        ban_radius_km = BAN_RADIUS_KM
    if vehicle_speed_kmph is None:
        vehicle_speed_kmph = DEFAULT_SPEED_KMPH
    
    start_dt = datetime.fromisoformat(start_datetime)
    if start_dt.tzinfo is None:
        start_dt = start_dt.replace(tzinfo=SAUDI_TZ)
    else:
        start_dt = start_dt.astimezone(SAUDI_TZ)
    
    client = openrouteservice.Client(key=ors_api_key, retry_over_query_limit=2)
    route = get_route_from_ors(client, start_lat, start_lon, end_lat, end_lon)
    
    geometry = route['features'][0]['geometry']['coordinates']
    segments = geometry
    
    ban_areas = []
    for _, row in ban_df.iterrows():
        ban_areas.append({
            'city': row['City'],
            'lat': float(row['Latitude']),
            'lon': float(row['Longitude']),
            'day_of_week': row['Day_of_Week'],
            'time_start': str(row['Time_Start']),
            'time_end': str(row['Time_End'])
        })
    
    delays = []
    current_time = start_dt
    last_point = segments[0]
    
    for i in range(1, len(segments)):
        p1 = last_point
        p2 = segments[i]
        
        seg_dist = haversine(p1[1], p1[0], p2[1], p2[0])
        seg_time = timedelta(hours=seg_dist / vehicle_speed_kmph)
        eta_to_seg = current_time + seg_time
        
        ban_hit = False
        for ban in ban_areas:
            in_ban = point_in_ban_area(p2[1], p2[0], ban['lat'], ban['lon'], ban_radius_km)
            eta_weekday = eta_to_seg.strftime('%A')
            
            if in_ban and eta_weekday == ban['day_of_week']:
                ban_start = datetime.combine(
                    eta_to_seg.date(), 
                    parse_time(ban['time_start']), 
                    tzinfo=SAUDI_TZ
                )
                ban_end = datetime.combine(
                    eta_to_seg.date(), 
                    parse_time(ban['time_end']), 
                    tzinfo=SAUDI_TZ
                )
                
                if ban_end <= ban_start:
                    ban_end += timedelta(days=1)
                
                if ban_start <= eta_to_seg <= ban_end:
                    wait = ban_end - eta_to_seg
                    delays.append({
                        'city': ban['city'],
                        'wait': wait,
                        'ban_start': ban_start,
                        'ban_end': ban_end,
                        'eta_at_ban': eta_to_seg,
                        'lat': ban['lat'],
                        'lon': ban['lon'],
                        'stop_lat': p2[1],
                        'stop_lon': p2[0]
                    })
                    current_time = ban_end
                    ban_hit = True
                    break
        
        if not ban_hit:
            current_time = eta_to_seg
        
        last_point = p2
    
    # Final check for end point
    for ban in ban_areas:
        in_ban = point_in_ban_area(end_lat, end_lon, ban['lat'], ban['lon'], ban_radius_km)
        eta_weekday = current_time.strftime('%A')
        
        if in_ban and eta_weekday == ban['day_of_week']:
            ban_start = datetime.combine(
                current_time.date(), 
                parse_time(ban['time_start']), 
                tzinfo=SAUDI_TZ
            )
            ban_end = datetime.combine(
                current_time.date(), 
                parse_time(ban['time_end']), 
                tzinfo=SAUDI_TZ
            )
            
            if ban_end <= ban_start:
                ban_end += timedelta(days=1)
            
            if ban_start <= current_time <= ban_end:
                wait = ban_end - current_time
                delays.append({
                    'city': ban['city'],
                    'wait': wait,
                    'ban_start': ban_start,
                    'ban_end': ban_end,
                    'eta_at_ban': current_time,
                    'lat': ban['lat'],
                    'lon': ban['lon'],
                    'stop_lat': end_lat,
                    'stop_lon': end_lon
                })
                current_time = ban_end
    
    # Build schedule
    schedule = []
    
    schedule.append({
        'vehicle_key': vehicle_key,
        'key': key,
        'event': 'start',
        'time': start_dt.strftime('%Y-%m-%d %H:%M'),
        'lat': start_lat,
        'lon': start_lon,
        'city': '',
        'wait_minutes': '',
        'ban_arrival': '',
        'ban_departure': ''
    })
    
    for d in delays:
        wait_minutes = int((d['wait'].total_seconds() + 59) // 60)
        schedule.append({
            'vehicle_key': vehicle_key,
            'key': key,
            'event': 'ban',
            'time': d['eta_at_ban'].strftime('%Y-%m-%d %H:%M'),
            'lat': d['stop_lat'],
            'lon': d['stop_lon'],
            'city': d['city'],
            'wait_minutes': wait_minutes,
            'ban_arrival': d['eta_at_ban'].strftime('%Y-%m-%d %H:%M'),
            'ban_departure': (d['eta_at_ban'] + d['wait']).strftime('%Y-%m-%d %H:%M')
        })
    
    schedule.append({
        'vehicle_key': vehicle_key,
        'key': key,
        'event': 'end',
        'time': current_time.strftime('%Y-%m-%d %H:%M'),
        'lat': end_lat,
        'lon': end_lon,
        'city': '',
        'wait_minutes': '',
        'ban_arrival': '',
        'ban_departure': ''
    })
    
    return {
        'eta': current_time,
        'schedule': schedule,
        'delays': delays,
        'route_segments': segments
    }

print("✅ Main calculation function defined!")

In [ ]:
def create_route_map(start_lat, start_lon, end_lat, end_lon, delays, segments, output_file="route_map.html"):
    """Create and save a folium map showing the route and ban stops."""
    mid_lat = (start_lat + end_lat) / 2
    mid_lon = (start_lon + end_lon) / 2
    m = folium.Map(location=[mid_lat, mid_lon], zoom_start=6)
    
    folium.PolyLine(
        [(lat, lon) for lon, lat in segments], 
        color="blue", 
        weight=5, 
        opacity=0.7
    ).add_to(m)
    
    folium.Marker(
        [start_lat, start_lon],
        popup=f"Start ({start_lat:.4f}, {start_lon:.4f})",
        icon=folium.Icon(color="green", icon="play")
    ).add_to(m)
    
    folium.Marker(
        [end_lat, end_lon],
        popup=f"End ({end_lat:.4f}, {end_lon:.4f})",
        icon=folium.Icon(color="red", icon="stop")
    ).add_to(m)
    
    for d in delays:
        wait_minutes = int((d['wait'].total_seconds() + 59) // 60)
        departure_time = (d['eta_at_ban'] + d['wait']).strftime('%Y-%m-%d %H:%M')
        folium.CircleMarker(
            location=[d['stop_lat'], d['stop_lon']],
            radius=14,
            color="orange",
            fill=True,
            fill_color="red",
            fill_opacity=0.95,
            tooltip=f"Ban Stop: {d['city']}",
            popup=folium.Popup(
                f"<b>Ban Stop: {d['city']}</b><br>"
                f"Arrival: {d['eta_at_ban'].strftime('%Y-%m-%d %H:%M')}<br>"
                f"Departure: {departure_time}<br>"
                f"Wait: {wait_minutes} min<br>"
                f"Ban Window: {d['ban_start'].strftime('%H:%M')} - {d['ban_end'].strftime('%H:%M')}",
                max_width=300
            )
        ).add_to(m)
    
    m.save(output_file)
    return output_file

def print_trip_results(result, start_lat, start_lon, end_lat, end_lon, start_datetime):
    """Print formatted trip results."""
    eta = result['eta']
    delays = result['delays']
    
    print("="*60)
    print(f"ESTIMATED TIME OF ARRIVAL: {eta.strftime('%Y-%m-%d %H:%M:%S %Z')}")
    print("="*60)
    
    if delays:
        print(f"\n🚫 {len(delays)} BAN AREA DELAY(S) ENCOUNTERED:")
        print("-" * 50)
        for i, d in enumerate(delays, 1):
            wait_minutes = int((d['wait'].total_seconds() + 59) // 60)
            print(f"{i}. City: {d['city']}")
            print(f"   Wait Time: {wait_minutes} minutes")
            print(f"   Arrival at Ban: {d['eta_at_ban'].strftime('%Y-%m-%d %H:%M')}")
            print(f"   Ban Window: {d['ban_start'].strftime('%H:%M')} - {d['ban_end'].strftime('%H:%M')}")
            print(f"   Location: ({d['stop_lat']:.4f}, {d['stop_lon']:.4f})")
            print()
    else:
        print("\n✅ NO BAN AREA DELAYS ENCOUNTERED")
    
    print("\n📍 DETAILED TRIP SCHEDULE:")
    print("-" * 50)
    
    start_dt = datetime.fromisoformat(start_datetime)
    if start_dt.tzinfo is None:
        start_dt = start_dt.replace(tzinfo=SAUDI_TZ)
    else:
        start_dt = start_dt.astimezone(SAUDI_TZ)
    
    print(f"🟢 START:   {start_dt.strftime('%Y-%m-%d %H:%M')} at ({start_lat:.4f}, {start_lon:.4f})")
    
    for d in delays:
        wait_minutes = int((d['wait'].total_seconds() + 59) // 60)
        print(f"🔴 BAN ARR: {d['eta_at_ban'].strftime('%Y-%m-%d %H:%M')} at ({d['stop_lat']:.4f}, {d['stop_lon']:.4f}) [{d['city']}] (wait {wait_minutes} min)")
        print(f"🟡 BAN DEP: {(d['eta_at_ban'] + d['wait']).strftime('%Y-%m-%d %H:%M')} at ({d['stop_lat']:.4f}, {d['stop_lon']:.4f}) [{d['city']}]")
    
    print(f"🏁 END:     {eta.strftime('%Y-%m-%d %H:%M')} at ({end_lat:.4f}, {end_lon:.4f})")
    print("="*60)

def display_schedule_table(schedule):
    """Display the schedule as a formatted table."""
    df = pd.DataFrame(schedule)
    display_cols = ['event', 'time', 'lat', 'lon', 'city', 'wait_minutes', 'ban_arrival', 'ban_departure']
    df_display = df[display_cols].copy()
    df_display = df_display.replace('', None)
    
    print("\n📊 SCHEDULE TABLE:")
    print("-" * 50)
    display(df_display)

print("✅ Display functions defined!")

In [ ]:
# Set up your API key
ORS_API_KEY =  "5b3ce3597851110001cf62483a1b4539f1a94e73b7ebee0c67f14d18"

print("🗺️  ETA Estimator with Ban Areas - Ready!")
print("=" * 60)
print("Instructions:")
print("1. Make sure you have 'ban_times.csv' in the same directory")
print("2. Set your OpenRouteService API key above")
print("3. Run the cell below with your trip parameters")
print("=" * 60)

In [ ]:
# TRIP PARAMETERS - MODIFY THESE VALUES FOR YOUR TRIP

# Trip coordinates
start_lat = 22.772041
start_lon =  39.068789
end_lat =26.483845
end_lon = 50.105058

# Trip timing
start_datetime = "2025-07-09T13:00:00"

# Optional parameters
vehicle_key = "VEHICLE_001"
key = "TRIP_001"
ban_radius_km = 25
vehicle_speed_kmph = 50

# EXECUTE THE TRIP CALCULATION
ORS_API_KEY = "5b3ce3597851110001cf62483a1b4539f1a94e73b7ebee0c67f14d18"

try:
    print(f"🚗 Calculating ETA for trip from ({start_lat}, {start_lon}) to ({end_lat}, {end_lon})")
    print(f"🕐 Starting at: {start_datetime}")
    print("🔧 Using hardcoded API key")
    
    result = calculate_eta_with_bans(
        start_lat, start_lon, 
        end_lat, end_lon,
        start_datetime, 
        ORS_API_KEY,
        vehicle_key=vehicle_key,
        key=key,
        ban_radius_km=ban_radius_km,
        vehicle_speed_kmph=vehicle_speed_kmph
    )
    
    print_trip_results(result, start_lat, start_lon, end_lat, end_lon, start_datetime)
    display_schedule_table(result['schedule'])
    
    print("\n🗺️  Generating route map...")
    map_file = create_route_map(
        start_lat, start_lon, 
        end_lat, end_lon,
        result['delays'], 
        result['route_segments'],
        "route_map.html"
    )
    print(f"✅ Route map saved as '{map_file}'")
    print("   Open this file in your browser to view the interactive route map!")
    
    try:
        print("\n🌐 Interactive Map Preview:")
        display(IFrame(map_file, width=800, height=600))
    except:
        print("   (Map preview not available - open the HTML file directly)")
        
except Exception as e:
    print(f"❌ Error: {e}")
    print("Please check your parameters and ensure ban_times.csv exists.")

print("\n" + "="*60)
print("✨ To calculate another trip, modify the parameters above and re-run this cell!")
print("="*60)

## How to Use This Notebook

1. **First time setup**: Run all cells above to import libraries and define functions
2. **Set API Key**: Replace `"YOUR_API_KEY_HERE"` with your OpenRouteService API key
3. **Configure trip**: Modify the parameters in the trip calculation cell
4. **Run calculation**: Execute the trip calculation cell
5. **View results**: The notebook will display ETA, delays, schedule table, and interactive map

## For New Trips
Simply modify the parameters in the trip calculation cell and re-run it!

In [ ]:
# OPTIONAL: Display Available Ban Areas

try:
    print("\n📋 Available Ban Areas:")
    print("-" * 50)
    ban_df = load_ban_areas(BAN_CSV)
    display(ban_df.head(10))
    print(f"Total ban areas loaded: {len(ban_df)}")
except Exception as e:
    print(f"⚠️  Could not load ban areas: {e}")